All imports:

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Define display function

In [115]:
def display_image(mat_like, cmap="rgb"):
    plt.figure()
    if cmap != "rgb":
        plt.imshow(mat_like, cmap=cmap)
    else:
        plt.imshow(mat_like)
    plt.show()
    plt.close()

load root folder and file location

In [116]:
current_folder = os.getcwd()
root_data_folder = os.path.join(current_folder, "local_data")
random_frames_folder = os.path.join(root_data_folder, "random_frames")
sequence_1_folder = os.path.join(root_data_folder, "sequence_1")
sequence_2_folder = os.path.join(root_data_folder, "sequence_2")
sequence_3_folder = os.path.join(root_data_folder, "sequence_3")
sequence_4_folder = os.path.join(root_data_folder, "sequence_4")

Clean image function

In [117]:
dil_kernel_size = 15
dil_kernel = cv2.getStructuringElement(
    cv2.MORPH_ELLIPSE, (dil_kernel_size, dil_kernel_size)
)


def clean_image(gray, display=False):
    gray = cv2.medianBlur(gray, 31)  # blur
    if display:
        display_image(gray, "gray")

    gray = cv2.equalizeHist(gray)  # equalize
    if display:
        display_image(gray, "gray")

    gray = 255 - gray  # inverse
    if display:
        display_image(gray, "gray")

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(13, 13))
    gray = clahe.apply(gray)  # clahe
    if display:
        display_image(gray, "gray")

    _, gray = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)  # threshold
    if display:
        display_image(gray, "gray")

    gray = cv2.dilate(gray, dil_kernel, iterations=3)  # dilate
    if display:
        display_image(gray, "gray")

    gray = cv2.erode(gray, dil_kernel, iterations=8)  # erode
    if display:
        display_image(gray, "gray")

    gray = cv2.dilate(gray, dil_kernel, iterations=6)
    if display:
        display_image(gray, "gray")

    return gray

find circles function

In [118]:
def get_circles_from_gray(gray, color=None):
    gray = clean_image(gray)
    # display_image(gray, "gray")
    circles = cv2.HoughCircles(
        gray,
        cv2.HOUGH_GRADIENT,
        dp=1,  # down sample size
        minDist=400,  # minimum distance between detected circles
        param1=9,
        param2=13,
        minRadius=50,
        maxRadius=100,
    )

    if circles is None:
        print("no matches")
        return []

    color_img_cpy = np.copy(color)
    circles = np.uint(np.around(circles))
    circles = circles[0, :]

    if color is None:
        return circles

    for circle in circles:
        x, y, r = circle[0], circle[1], circle[2]
        cv2.circle(color_img_cpy, (x, y), r, (255, 255, 255), 15)
    # display_image(color_img_cpy)
    return circles


def merge_circle_arr(res, add):
    for circle_new in add:
        x_new, y_new, r_new = circle_new[0], circle_new[1], circle_new[2]
        for old_circle_dict in res:
            x_old = old_circle_dict["x"]
            y_old = old_circle_dict["y"]
            r_old = old_circle_dict["radius"]
            
            p1 = np.array([x_new, y_new], dtype=np.int64)
            p2 = np.array([x_old, y_old], dtype=np.int64)
            values = p1 - p2
            distance = np.linalg.norm(values)
            if distance < 100:
                x_new = (x_new + x_old) // 2
                y_new = (y_new + y_old) // 2
                r_new = np.max(np.array([r_new, r_old]))
                
                old_circle_dict["x"] = x_new
                old_circle_dict["y"] = y_new
                old_circle_dict["radius"] = r_new
                r_new = -1
                break

        if r_new > 0:
            circle_dict = {}
            circle_dict["x"] = x_new
            circle_dict["y"] = y_new
            circle_dict["radius"] = r_new
            res.append(circle_dict)


def get_colored_circles(color, display=False):
    r, g, b = cv2.split(color)

    r_circles = get_circles_from_gray(r, color)
    g_circles = get_circles_from_gray(g, color)
    b_circles = get_circles_from_gray(b, color)

    result = []
    merge_circle_arr(result, r_circles)
    merge_circle_arr(result, g_circles)
    merge_circle_arr(result, b_circles)

    if not display:
        return result

    img_copy = np.copy(color)
    for circle in result:
        x, y, r = circle["x"], circle["y"], circle["radius"]
        cv2.circle(img_copy, (x, y), r, (255, 255, 255), 15)

    display_image(img_copy)
    return result

load one image for testing

In [119]:
orange = np.array([123, 85, 66], dtype=np.int64)
light_blue = np.array([60, 76, 101], dtype=np.int64)
gray = np.array([60, 70, 70], dtype=np.int64)
green = np.array([93, 111, 70], dtype=np.int64)
color_values = [
    orange, light_blue, gray, green
]
color_names = [
    "red", "blue", "gray", "green"
]

def get_color_name(color):
    color_int = np.array(color, dtype=np.int64)
    min_distance =  np.linalg.norm(color_int - color[0])
    min_color_idx = 0
    
    for i in range(len(color_values)):
        distance = np.linalg.norm(color - color_values[i])
        if distance < min_distance:
            min_color_idx = i
            min_distance = distance
    
    return color_names[min_color_idx]
    

def detect_colored_circles(img_path):
    rnd_img = cv2.imread(img_path)
    color_rgb = cv2.cvtColor(rnd_img, cv2.COLOR_BGR2RGB)
    circles = get_colored_circles(color_rgb)
    
    for circle in circles:
        x, y, r = circle["x"], circle["y"], circle["radius"]
        mask = np.zeros(color_rgb.shape[:2], dtype=np.uint8)
        cv2.circle(mask, (x, y), r // 2, 255, thickness=-1)
        mean_color = cv2.mean(color_rgb, mask=mask)
        mean_color = np.int64(np.around(mean_color))[:3]
        circle["color"] = get_color_name(mean_color)
        
    return circles

img_path = os.path.join(sequence_4_folder, "seq_000.jpg")

print(detect_colored_circles(img_path))

[{'x': np.uint64(444), 'y': np.uint64(873), 'radius': np.uint64(86), 'color': 'gray'}, {'x': np.uint64(1187), 'y': np.uint64(106), 'radius': np.uint64(76), 'color': 'gray'}, {'x': np.uint64(1327), 'y': np.uint64(982), 'radius': np.uint64(83), 'color': 'green'}, {'x': np.uint64(810), 'y': np.uint64(488), 'radius': np.uint64(75), 'color': 'blue'}, {'x': np.uint64(1718), 'y': np.uint64(600), 'radius': np.uint64(90), 'color': 'red'}]


Now starting part 2 of the exercise!

In [120]:
def track_circles_over_time(img_paths):
    circles_from_images = []
    for img_path in img_paths:
        circles_from_images.append(detect_colored_circles(img_path))
    
    result = []
    for idx in range(len(circles_from_images)):
        circles = circles_from_images[idx]
        circle_id = len(result)
        for circle in circles:
            x_new = circle["x"]
            y_new = circle["y"]
            
            for res in result:
                x_old = res["x"]
                y_old = res["y"]
                
                p1 = np.array([x_new, y_new], dtype=np.int64)
                p2 = np.array([x_old, y_old], dtype=np.int64)
                values = p1 - p2
                distance = np.linalg.norm(values)
                if distance < 100:
                    circle_id = res["circle_id"]
                    break

            new_res = {}
            new_res["image_id"] = idx + 1
            new_res["circle_id"] = circle_id
            new_res["x"] = circle["x"]
            new_res["y"] = circle["y"]
            new_res["radius"] = circle["radius"]
            new_res["color"] = circle["color"]
            result.append(new_res)
    
    return result
        
        
img_lst = os.listdir(sequence_2_folder)
img_paths = [os.path.join(sequence_1_folder, img_fn) for img_fn in img_lst]
print(track_circles_over_time(img_paths))

[{'image_id': 1, 'circle_id': 0, 'x': np.uint64(508), 'y': np.uint64(455), 'radius': np.uint64(95), 'color': 'blue'}, {'image_id': 1, 'circle_id': 0, 'x': np.uint64(87), 'y': np.uint64(863), 'radius': np.uint64(92), 'color': 'gray'}, {'image_id': 1, 'circle_id': 0, 'x': np.uint64(1040), 'y': np.uint64(1018), 'radius': np.uint64(75), 'color': 'red'}, {'image_id': 1, 'circle_id': 0, 'x': np.uint64(1476), 'y': np.uint64(615), 'radius': np.uint64(96), 'color': 'red'}, {'image_id': 2, 'circle_id': 0, 'x': np.uint64(505), 'y': np.uint64(457), 'radius': np.uint64(96), 'color': 'blue'}, {'image_id': 2, 'circle_id': 0, 'x': np.uint64(93), 'y': np.uint64(861), 'radius': np.uint64(93), 'color': 'gray'}, {'image_id': 2, 'circle_id': 0, 'x': np.uint64(1044), 'y': np.uint64(1014), 'radius': np.uint64(84), 'color': 'red'}, {'image_id': 2, 'circle_id': 0, 'x': np.uint64(1470), 'y': np.uint64(607), 'radius': np.uint64(85), 'color': 'red'}, {'image_id': 3, 'circle_id': 0, 'x': np.uint64(88), 'y': np.uin